In [11]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

In [28]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [13]:
# Create a loader instance
loader = HuggingFaceDatasetLoader('Q&A_format.md')

# Load the data
with open('Q&A_format.md', 'r', encoding='utf-8') as file:
    data = file.read()

# Display the first 100 entries
data[:100]

'Question: What is a Reservoir Computing architecture?\nAnswer: A Reservoir Computing (RC) architectur'

In [30]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
#docs = text_splitter.split_documents(data)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(data)

AttributeError: 'str' object has no attribute 'page_content'

In [21]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

c:\Users\arthu\AppData\Local\Programs\Python\Python39\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\arthu\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\arthu\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-l6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
query_result = embeddings.embed_query(data)
query_result

[-0.06041721999645233,
 -0.057492200285196304,
 -0.028663907200098038,
 -0.003117320127785206,
 -0.018313992768526077,
 0.09196006506681442,
 0.030595237389206886,
 -0.04750973358750343,
 0.008088894188404083,
 -0.08457435667514801,
 -0.09845414757728577,
 0.001594462781213224,
 0.013694314286112785,
 -0.014953358098864555,
 -0.12278381735086441,
 0.024595320224761963,
 0.030579328536987305,
 -0.049063537269830704,
 -0.015709377825260162,
 -0.04961153492331505,
 0.04865964502096176,
 0.010904169641435146,
 -0.053949736058712006,
 0.01784193329513073,
 -0.003001793287694454,
 0.11723509430885315,
 -0.03335120901465416,
 -0.04590585082769394,
 -0.03861362859606743,
 0.016017120331525803,
 0.04525875672698021,
 -0.02180538699030876,
 0.010311756283044815,
 0.008703208528459072,
 0.04830273240804672,
 0.06430995464324951,
 -0.06577448546886444,
 -0.026444779708981514,
 -0.04999789595603943,
 -0.015338215976953506,
 0.05403302609920502,
 -0.012027174234390259,
 0.035131122916936874,
 0.0237

In [26]:
db = FAISS.from_documents(data, embeddings)

AttributeError: 'str' object has no attribute 'page_content'